# Citations with Claude

## Setup

To complete the following guide you will need to install the `anthropic` package and obtain and Anthropic API key. 

In [ ]:
!pip install anthropic pypdf

In [1]:
import anthropic
import os

client = anthropic.Anthropic(
    api_key='' # YOUR API KEY HERE,
)

## Overview

Large Language Models (LLMs) like Claude have demonstrated remarkable capabilities in understanding and generating human-like text across a wide range of applications. However, one of the challenges with LLMs is ensuring the accuracy and explainability of the information they provide. This is where citations become invaluable.

Citations are references to specific sources of information that support or substantiate the claims or statements made by the LLM. By incorporating citations into Claude's responses, we can significantly enhance the transparency and usefulness of the Claude-generated content.

### The Value of Citations

**Verifiability and User Affordance**: Citations provide clickable links or clear references to source material, allowing end users to verify information independently and explore topics further. This enhances credibility and empowers users to access additional context easily.

**Explainability**:
By linking responses to specific sources, citations make Claude's reasoning process more transparent. This aids in building trust and provides valuable metadata for analysis and quality control. For instance, in a voice-based AI assistant, it might not make sense to verbalize the citations. Instead, they could be logged separately for analysis or quality assurance purposes.

### Practical Applications

In this guide, we will explore how to effectively use citations with Claude in two specific Q&A scenarios:

1. Answering queries over a help center knowledge base
2. Extracting information from call transcripts

Through these examples, you'll learn how to prompt Claude to provide citations, how to process and present these citations, and how to evaluate them to enhance the overall quality and trustworthiness of your AI-powered applications.



## Use case: Q&A against customer help center

Question-and-Answer (Q&A) systems built on top of customer help centers are a popular application of Claude. These systems allow customers to quickly find answers to their questions without having to manually search through multiple help articles. Importantly, these systems often require citations, as they provide direct links to the relevant help articles, allowing users to verify the information and dive deeper into topics if needed.


For our example, let's introduce "PetWorld", a fictional e-commerce company that specializes in pet supplies and accessories. PetWorld has a wide range of products for various pets, from dogs and cats to reptiles and small mammals. They've been growing rapidly and want to improve their customer support by implementing an AI-powered chatbot that can answer customer queries by referencing their extensive online help center.


PetWorld's help center covers a variety of topics, including:

- Product information and care instructions
- Shipping and delivery policies
- Returns and exchanges
- Account management
- Pet care tips and advice

By implementing a citation-based Q&A system, PetWorld aims to:

- Reduce the workload on their customer service team
- Provide 24/7 support to their customers
- Ensure customers can easily find and verify information in their help center
- Improve customer satisfaction by providing quick, accurate answers with links to further reading

In the following sections, we'll explore how to build such a system using Claude, enabling PetWorld to create a more efficient and user-friendly customer support experience.

### Help Center

In this example we provide a ficitonal help center comprising of 10 articles. Each article includes a title and the body of text. We will use these articles to build and test a prompt which is able to answer user questions based on these articles and cite the response.

Note: In this example we will not explore retrieving specific articles from the help center. However, in many partical applications a help center may consist of dozens or hundreds of articles. Often times this will require implementing more complex retrieval (RAG) which will be covered in a separate skills cookbook.

All articles can be within in this cookbook under `data/help_center_articles`

In [2]:
import os

# Directory containing the help center articles
articles_dir = 'data/help_center_articles'
filenames = sorted([f for f in os.listdir(articles_dir) if f.endswith('.txt')])

for filename in filenames:
    file_path = os.path.join(articles_dir, filename)
    with open(file_path, 'r') as file:
        title = file.readline().strip()
    # Print the filename and title
    print(f"({filename}) {title}")

(0.txt) title: How to Change Your Password
(1.txt) title: 30-Day Return Policy
(2.txt) title: How to Change Delivery Address
(3.txt) title: Order Tracking Information
(4.txt) title: Rewards Program: Points System
(5.txt) title: Bulk Order Discounts
(6.txt) title: Shipping Options and Policies
(7.txt) title: Cancelling a Subscription
(8.txt) title: Account Creation and Management
(9.txt) title: Pet Insurance Overview


### Create a prompt for conversational Q&A

In this example we will create a prompt which takes a **user question** and the **help center articles** as inputs and return a helpful response. In this prompt template we apply several best practices including assigning a role and providing an example. Review the Anthropic [prompt engineering documentation](https://docs.anthropic.com/en/docs/prompt-engineering) to learn more.


In [3]:
sysyem_prompt = "You will be acting as a conversational AI customer support assistant for the ecommerce website PetWorld. Your goal is to help answer customer questions in a friendly and helpful manner, using PetWorld's help center articles as your knowledge base."

prompt_template = '''
Here are the help center articles you have available, provided in <article> tags with unique IDs:

<help_center_articles>
{HELP_CENTER_ARTICLES}
</help_center_articles>

And here is the user's question, provided in a <user_question> tag:

<user_question>
{USER_QUESTION}
</user_question>

To formulate your response, follow these steps:

1. Carefully read the user's question to understand what they are asking about. 
2. Search through the provided help center articles to find the most relevant information to answer the question. Focus on finding an article that directly addresses the user's specific question.
3. If you find a relevant article, use the information in it to write a friendly response that fully answers the user's question. Aim to provide a complete answer using only information from the help center articles.
4. At the end of your response, include a citation like this - [Article ID] - where "Article ID" is replaced by the ID number of the help center article you used to answer the question.
5. If after searching the help center articles you determine that none of them contain the information needed to answer the user's question, simply respond with "I'm afraid I don't know the answer to that question. Let me know if there is anything else I can assist with!"

Here is an example of what a good response looks like:

<user_question>What is your return policy on dog food?</user_question>

<answer>At PetWorld, we offer a 30 day return window on all dog food purchases. You can return the unused portion for a full refund within 30 days of purchase. We also offer a 100% satisfaction guarantee - if your dog doesn't love their food, we'll give you your money back! Let me know if you have any other questions. [1]</answer>

Now it's your turn! Please provide your response to the user's question inside <answer> tags. Remember - only use information from the provided help center articles, and if you can't find the answer there, let the user know you don't have that information. Always aim to be friendly and helpful in your tone.'''

Next we will apply preprocessing to the help center articles

In [4]:
def get_articles_as_string():
    # Get all .txt files in the directory and sort them
    filenames = sorted([f for f in os.listdir(articles_dir) if f.endswith('.txt')])

    # String to hold all articles
    all_articles = ""

    # Iterate through the sorted list of files
    for filename in filenames:
        file_path = os.path.join(articles_dir, filename)
        with open(file_path, 'r') as file:
            # Read the entire content of the file
            content = file.read().strip()

        # Split the content into title and body
        parts = content.split('\n', 1)
        title = parts[0].strip()
        body = parts[1].strip() if len(parts) > 1 else ""

        # Remove "title:" prefix if it exists
        if title.lower().startswith("title:"):
            title = title[6:].strip()
        
        # Strip .txt from filename for the id
        article_id = filename[:-4] if filename.endswith('.txt') else filename

        # Format the article
        article = f"""<article id="{article_id}">
<title>{title}</title>
<content>{body}</content>
</article>
"""
        all_articles += article

    return all_articles

# Get the formatted string of all articles
articles_string = get_articles_as_string()

# Print first and last 1000 characters of the articles string
print(f'{articles_string[:1000]}\n...\n{articles_string[-1000:]}')

<article id="0">
<title>How to Change Your Password</title>
<content>To change your password, log in to your PetWorld account and navigate to the "Account Settings" page. Click on the "Security" tab, then select "Change Password." Enter your current password, then type your new password twice to confirm. Make sure your new password is at least 8 characters long and includes a mix of uppercase and lowercase letters, numbers, and symbols. Click "Save Changes" to update your password. For security reasons, you'll be logged out and need to sign in again with your new password.</content>
</article>
<article id="1">
<title>30-Day Return Policy</title>
<content>PetWorld offers a 30-day return policy on most items. If you're not satisfied with your purchase, you can return it within 30 days of the delivery date for a full refund or exchange. The item must be unused, in its original packaging, and in resalable condition. To initiate a return, log in to your account, go to "Order History," selec

Finally, we will call Claude 3.5 Sonnet with the prompt template, our formatted help center articles, and the user question.

In [5]:
def answer_question(user_question):
    # Generate the prompt with the user question and help center articles
    prompt = prompt_template.format(HELP_CENTER_ARTICLES=articles_string, USER_QUESTION=user_question)

    response = client.messages.create(
        model='claude-3-5-sonnet-20241022',
        system=sysyem_prompt,
        messages=[
            {
                'role': 'user',
                'content': prompt
            },
            {
                'role': 'assistant',
            'content': '<answer>'
            }

        ],
        max_tokens=2000,
        stop_sequences=['</answer>'],
        temperature=0
    )
    return response.content[0].text


raw_output = answer_question('Confused how to change my address')
print(raw_output)


I'll be happy to help you change your delivery address! To update an address for an order, here's what you need to do:

1. Log in to your PetWorld account
2. Go to "Order History"
3. Find the order you want to modify
4. Click "Change Shipping Address"
5. Enter your new address and save the changes

Please note that you can only change the address if your order hasn't shipped yet. If your order is already in transit, you'll need to contact our customer support team for help.

For future orders, you can also add or edit addresses in your account settings under "Saved Addresses" - this way you'll have them ready to use for your next purchase!

Is there anything else I can help you with? [2]


### Post processing and running

When working with citations, it's often beneficial to apply post-processing techniques to the output. One useful approach is to add URLs and other metadata to the citations after Claude has generated the response. This method offers several advantages:

- It saves on output tokens, as Claude doesn't need to generate full URLs.
- It limits the possibility of errors in URL generation.
- It provides flexibility in how citations are presented to the end-user.

In our PetWorld example, let's say each article ID maps to a specific URL on their help center website. For instance:

- `0.txt` maps to `https://help.petworld.com/article/0`
- `1.txt` maps to `https://help.petworld.com/article/1`
- And so on...

We can apply a post-processing step to Claude's output to turn these citations into clickable hyperlinks. Here's a Python function that could accomplish this:

In [6]:
import re
from IPython.display import HTML, display

def process_citations(text):
    def replace_citation(match):
        article_id = match.group(1)
        url = f"https://help.petworld.com/article/{article_id}"
        return f'<a href="{url}">[{article_id}]</a>'

    pattern = r'\[(\d+)\]'
    return re.sub(pattern, replace_citation, text)


post_processed_output = process_citations(raw_output)
display(HTML(f'<pre>{post_processed_output}</pre>'))

### Evaluation
Now that we have a working prompt end-to-end we can measure and evaluate Claude's ability to correct cite sources in this task. This is separate from evaluating the answer quality itself which will be covered in a separate cookbook.

To evaluate citations you will need to create a golden set of (question, article ID) pairs. You will be evaluating that given a user question Claude is citing the expected article ID(s). In the evaluation we also test questions that are not answered by the help center and confirm the response does not include a citation.

We will use `promptfoo` for this portion. Head over to `evaluation/README.md` to get started.


## Use case: Q&A over a Large Document

In our previous example, we explored how to prompt Claude for Q&A over a help center, providing citations at the article level. This approach is ideal when dealing with numerous short articles, as it allows end users to easily read entire referenced articles.

However, when working with larger documents, a different citation strategy becomes necessary. For instance, if you're dealing with a 40-page document, a citation referencing the entire document provides little value to the user. In such cases, it's more beneficial to cite specific quotes or passages that directly inform the answer.

In this section, we'll develop a prompt that enables Claude to perform Q&A over a large document while extracting and citing relevant quotes. This approach offers several advantages:

- Precision: It pinpoints the exact information used to formulate the answer.
- Verifiability: Users can quickly check the source of specific claims.
- Context: It provides users with the surrounding context of the information.
- Efficiency: It saves users time by highlighting the most relevant parts of a large document.

### Preprocessing

In this example we will work with a large PDF, "Constitutional AI: Harmlessness from AI Feedback" published by Anthropic. This document is 34 pages. Since the API does not natively ingest files in PDF format we will apply some preprocessing to extract the text.

In [7]:
from pypdf import PdfReader
reader = PdfReader("./data/Constitutional AI.pdf")
paper_text = ''.join(page.extract_text() for page in reader.pages)
print(paper_text[:1000])

Constitutional AI: Harmlessness from AI Feedback
Yuntao Bai∗, Saurav Kadavath, Sandipan Kundu, Amanda Askell, Jackson Kernion,
Andy Jones, Anna Chen, Anna Goldie, Azalia Mirhoseini, Cameron McKinnon,
Carol Chen, Catherine Olsson, Christopher Olah, Danny Hernandez, Dawn Drain,
Deep Ganguli, Dustin Li, Eli Tran-Johnson, Ethan Perez, Jamie Kerr, Jared Mueller,
Jeffrey Ladish, Joshua Landau, Kamal Ndousse, Kamile Lukosuite, Liane Lovitt,
Michael Sellitto, Nelson Elhage, Nicholas Schiefer, Noemi Mercado, Nova DasSarma,
Robert Lasenby, Robin Larson, Sam Ringer, Scott Johnston, Shauna Kravec,
Sheer El Showk, Stanislav Fort, Tamera Lanham, Timothy Telleen-Lawton, Tom Conerly,
Tom Henighan, Tristan Hume, Samuel R. Bowman, Zac Hatﬁeld-Dodds, Ben Mann,
Dario Amodei, Nicholas Joseph, Sam McCandlish, Tom Brown, Jared Kaplan∗
Anthropic
Abstract
As AI systems become more capable, we would like to enlist their help to supervise
other AIs. We experiment with methods for training a harmless AI assistant

### Prompt

Our prompt template will be similar to the previous example. We apply the same prompting best practices including beginning with a role, following with document contents wrapped in XML, and finishing with the full instructions. We include additional instructions to describe the quote extraction process and describe the desired output format.


In [8]:
system_prompt = "You are an AI research assistant. Your task is to provide detailed answers to questions related to the content of the provided paper."
prompt_template = '''
Here is the paper you will be working with:
<paper>
{PAPER_CONTENT}
</paper>

And here is the user's question, provided in a <user_question> tag:
<user_question>
{USER_QUESTION}
</user_question>

To formulate your response, follow these steps:
1. Find the quotes from the paper that are the most relevant to answering the question. These quotes can be quite long if necessary (even multiple paragraphs). You may need to use many quotes to answer a single question, including code snippits and other examples.
2. Assign numbers to these quotes in the order they were found.
3. Based on the document and quotes, answer the question. Directly quote the documentation when possible, including examples.
4. When answering the question provide citations references in square brackets containing the number generated in step 2 (the number the citation was found)
5. Structure the output in the following format. Provide no preable or postamble:
<citations>
{{
   "citations": [
      {{
         
         "number": "integer",
         "passage": "string"
      }},
      ...
   ]
}}
</citations>

<answer>A plain text answer, formatted as Markdown[1]</answer>"""

Now it's your turn. First find and output the relevant quotes in the format described. Then provide your response to the user's question inside <answer> tags. Remember - only use information from the provided paper, and if you can't find the answer there, let the user know you don't have that information.'''

Now we call 3.5 Sonnet with the populated prompt template and output the response 

In [9]:
def answer_question(user_question):
    # Generate the prompt with the user question and help center articles
    prompt = prompt_template.format(PAPER_CONTENT=paper_text, USER_QUESTION=user_question)

    response = client.messages.create(
        model='claude-3-5-sonnet-20241022',
        system=system_prompt,
        messages=[
            {
                'role': 'user',
                'content': prompt
            },
            {
                'role': 'assistant',
            'content': '<citations>'
            }

        ],
        max_tokens=2000,
        stop_sequences=['</answer>'],
        temperature=0
    )
    return response.content[0].text


raw_output = answer_question('What is the main idea of the paper?')
print(raw_output)


{
   "citations": [
      {
         "number": 1,
         "passage": "As AI systems become more capable, we would like to enlist their help to supervise other AIs. We experiment with methods for training a harmless AI assistant through self-improvement, without any human labels identifying harmful outputs. The only human oversight is provided through a list of rules or principles, and so we refer to the method as 'Constitutional AI'."
      },
      {
         "number": 2,
         "passage": "The process involves both a supervised learning and a reinforcement learning phase. In the supervised phase we sample from an initial model, then generate self-critiques and revisions, and then finetune the original model on revised responses. In the RL phase, we sample from the finetuned model, use a model to evaluate which of the two samples is better, and then train a preference model from this dataset of AI preferences."
      },
      {
         "number": 3,
         "passage": "As a resul

### Post processing

The structured output format we've developed, combining specific citations with the AI's answer, offers significant advantages in working with large documents. By providing precise quotes and their locations, we enable users to quickly verify information without reading the entire source material. It is possible to build a UX on top of this approach where readers can easily access and validate relevant portions of the document. Ultimately, this method enhances the transparency and utility of AI-assisted Q&A systems, making large documents more accessible and their content more verifiable.